# Downloader and playlist creator of songs from Tracks

## Step 1 
Run `tracks-downloader.py` to get all tracks from the website. Results in 24148 entries in `tracks-db.txt`.

## Step 2
Run `tracks-db-remove-duplicates.py` to create `tracks-db-no-duplicates.txt` with 6805 entries.

## Step 3
Get OAuth Token from Spotify.
- [Bored of Libraries? Here’s How to Connect to the Spotify API Using Pure Python](https://python.plainenglish.io/bored-of-libraries-heres-how-to-connect-to-the-spotify-api-using-pure-python-bd31e9e3d88a) and [SpotifyAPIConnection](https://github.com/SeanDuttonJones/SpotifyAPIConnection)
- [Spotify Authorization Code Flow](https://developer.spotify.com/documentation/general/guides/authorization/code-flow/)